Let's evaluate a problem instance with 64x64-sized maps.

In [1]:
from neural_astar.utils.data import create_dataloader
import jax
import jax.numpy as jnp

dataloader = create_dataloader("../planning-datasets/data/mpd/all_064_moore_c16.npz", "test", 1)
map_design, start_map, goal_map, opt_traj = next(iter(dataloader))

# Convert variables into jax.numpy.array
map_design_j = jnp.array(map_design)[:, 0]
start_map_j = jnp.array(start_map)[:, 0]
goal_map_j = jnp.array(goal_map)[:, 0]

/home/yonetani/programs/neural-astar-jax/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of Test Samples: 400
	Size: 64x64


Here we use `VanillaAstar` that consists of only differentiable astar.

In [2]:
from neural_astar.planner.astar import VanillaAstar as VanillaAstarPyTorch

va_pytorch = VanillaAstarPyTorch()
%timeit va_pytorch(map_design, start_map, goal_map)

/home/yonetani/programs/neural-astar-jax/.venv/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


199 ms ± 3.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
from neural_astar_jax.planner.astar import VanillaAstar as VanillaAstarJax

va_jax = VanillaAstarJax()
# model initialization
variables = va_jax.init(jax.random.PRNGKey(0), map_design_j, start_map_j, goal_map_j)
# and jit-compiling forward function
va_jax_apply = jax.jit(va_jax.apply)

%timeit va_jax_apply(variables, map_design_j, start_map_j, goal_map_j)

70.9 ms ± 738 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
